#### python的实现(Implementation for add/multiplication)
> sum rule(和式法则) and product rule(乘式法则)
```math
\frac{\partial(a+b)}{\partial a}=\frac{\partial a}{\partial a}+\frac{\partial b}{\partial a} = 1 + 0 = 1
```
```math
\frac{\partial(ab)}{\partial a}=a*\frac{\partial b}{\partial a}+b*\frac{\partial b}{\partial b} = 0 + b = b
```

#### 常见导数的值
函数 | 原函数| 导函数
---|---|---
常函数 |  y=c（C为常数） | 0
指数函数 | y=e^x, y=a^x | y'=e^x,y=a^x*lna
幂函数 | y=x^n | y'=n*x^(n-1)


- 加法的实现规则
> 加法的反向传播并不需要保存值，对于输入参数的导数始终为1
- 乘法的实现规则：需保存输入值

```code
class AddLayer:
    def __init__(self):
        pass
    def forwardflow(self, a, b):
        c = a + b
        return c
    def backwardflow(self, rdiff):
        ra = rdiff * 1
        rb = rdiff * 1
        return ra, rb

class MulLayer:
    def __init__(self):
        self.a = 0
        self.b = 0
        self.c = 0
    def forwardflow(self, a, b):
        c = a * b
        self.a = a
        self.b = b
    def backwardflow(self, rdiff):
        ra = rdiff * self.b
        rb = rdiff * self.a
        return ra, rb
```

#### 各个重要激活函数的计算图和代码实现
1. sigmoid
```code
def sigmoidfunc(x):
    return 1/(1+np.exp(-x))

class Sigmoid:
    def __init__(self):
        self.result = None
    def forward(self, x):
        result = sigmoidfunc(x)
        self.result = result
        return result

    def backward(self, rdiff):
        ra = rdiff * (1.0 - self.result) * self.result
```
2. ReLU
> 考虑传入的参数为一个矩阵/数组
```code
#ReLu
#be careful the input/output array should be the same dimension
def ReLUfunc(x):
    return np.maximum(0, x)
class ReLu:
    def __init__(self):
        self.rmask = None
    def forwardflow(self, x):
        result = ReLUfunc(x)
        print(result)
        #array value pass(treat it as an object): python skill
        self.rmask = (x <= 0)
        print(self.rmask)
        return result
    def backwardflow(self, rdiff):
        rdiff[self.rmask] = 0
        return rdiff
```

#### Affine/softmax 实现和相关问题
##### Affine/仿射变化
> 想象在一个平面坐标系的情况
1. 一次线性：加权重运算（*运算）
2. 一次平移：加偏置运算（+运算）
```code
class Affine:
    def __init__(self):
        self.x = 0
        self.w = 0
        self.b = 0
    def forwardflow(self, x, b, w):
        self.w = w
        self.x = x
        return np.dot(x, b) + w
    def backwardflow(self, rdiff):
        dwx = rdiff
        db = np.sum(rdiff, axis=0)
        rw = np.dot(self.x.T, ridff)
        rx = np.dot(rdiff, self.w.T)
        return rx
```

> 关于行列式相乘的问题：
```math
\frac{\partial L}{\partial Y}*W^T=\frac{\partial L}{\partial X}
```
这里我们假设：

(dl/dy).shape = (3,)

W^T.shape = (2,3)

(dl/dx) = (3,)

实际补全的结果时：(1,3) * (2,3) = (1,3)
结果的行由被乘数确定，列由乘数确定。

##### 交叉熵误差（Softmax-with-Loss）

